In [1]:
import os
import sys
path_to_this_notebook = os.path.abspath('.')
path_to_project = path_to_this_notebook[:path_to_this_notebook.find('note')]
sys.path.append(path_to_project)
from src.save_and_load_tools.save_load_utils import *
from src.grid.grid_env import GridEnv
from src.measures import *
from src.plotting_tools.plot_performance import plot_performance
from src.planners.socp.socp_planner import SOCPPlanner
import re
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np
from src.simulation_tools.simulate import simulate
from src.grid.grid_env import GridEnv

%load_ext autoreload
%autoreload 2

In [2]:
def load_and_compute_metrics(experiment_name, path_to_experiments, base_names_list=None, ptu_int=60):
    
    results = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))
    if base_names_list is None:
        base_names_list = os.listdir(path_to_experiments + '/%s/' % experiment_name)
    for grid_name in base_names_list:
        #print('Spec', base_name)
        path_to_results_dict = path_to_experiments + '/%s/%s/results/' % (experiment_name, grid_name)
        scenarios, scenario_generator = load_scenarios(experiment_name, grid_name, path_to_experiments, ptu_int)
        grid = load_grid(experiment_name, grid_name, path_to_experiments)
        print(grid.n_loads, grid.n_gens, [g.p_min for g in grid.gens])
        #grid_id = 'p-gen-min=%d_i-line-max=%d' % (gen_p_min, line_i_max)
        #grid_name = grid.name[: first_ + 1] + grid_id + grid.name[first_:]
        grid.name = grid_name
        print(grid_name)
        results_dict_names = [f for f in os.listdir(path_to_results_dict) if '_results.pickle' in f]
        
        for rd_name in results_dict_names:
            pl_name = rd_name[:rd_name.find('results') - 1]
            pl_results_dict = load_results_dict(pl_name, experiment_name, grid_name, path_to_experiments)
            
            for sc_ind, sc_result in pl_results_dict.items():
                scenario = scenarios[sc_ind]
                price = np.max(scenario.power_price)
                w = measure_wealth(sc_result, scenario, grid)
                soc = measure_soc_percentage(sc_result, scenario)
                evs_charged = measure_charged_evs(sc_result, scenario, grid, .99)
                #n_lines_at_limit = measure_lines_at_limit(sc_result, scenario, grid, average=True, gap=0.01)
                #p_missmatch = measure_loads_misstmatch(sc_result, scenario, grid, average=True, gap=0.99)
                pl_time = sc_result['planning time'].mean()
                for key, val in sc_result.items():
                    results[grid.name][pl_name][key][sc_ind] = val
                evs_active = np.zeros((len(scenario.evs), len(scenario.timesteps_hr)))
                for ev_ind, ev in enumerate(scenario.evs):
                    t_arr_ind = np.where(scenario.timesteps_hr == ev.t_arr_hr)[0][0]
                    t_dep_ind = np.where(scenario.timesteps_hr == ev.t_dep_hr)[0][0]
                    evs_active[ev_ind, t_arr_ind: t_dep_ind] = 1
                    #print(ev.t_arr_hr, t_arr_ind)
                #results[grid.name][pl_name]['Plans missmatch'][sc_ind] = p_missmatch
                #results[grid.name][pl_name]['Lines at limit'][sc_ind] = n_lines_at_limit / grid.n_lines
                results[grid.name][pl_name]['Price'][sc_ind] = price
                results[grid.name][pl_name]['Wealth'][sc_ind] = w
                results[grid.name][pl_name]['Social welfare'][sc_ind] = w
                results[grid.name][pl_name]['SOC'][sc_ind] = soc
                results[grid.name][pl_name]['EVs charged'][sc_ind] = evs_charged
                results[grid.name][pl_name]['Pl. time'][sc_ind] = pl_time
                results[grid.name][pl_name]['EVs_active'][sc_ind] = evs_active
    return results

planner_names_dict = \
{\
 'SOCPPlanner_normalize=True_future=known-future_ev-locations=full_None': 'SOCP_full',
 'SOCPPlanner_normalize=True_future=known-future_ev-locations=present_None': 'SOCP_present',
 'SOCPPlanner_normalize=True_future=known-future_ev-locations=past_None': 'SOCP_past',
 'SOCPPlanner_normalize=True_future=known-future_ev-locations=blind_None': 'SOCP_blind',
 'SOCPPlanner_normalize=True_future=known-future_ev-locations=full_single-node': 'SOCP_single-node',
 'SOCPPlanner_normalize=True_future=known-future_ev-locations=full_single-node-unified': 'SOCP_single-node_uni',
 'SOCPPlanner_normalize=True_future=known-future_ev-locations=full_single-node-weird': 'SOCP_single-node_weird',
 'SOCPPlanner_normalize=True_future=known-future_ev-locations=full_parallel': 'SOCP_parallel',
 'SOCPPlanner_normalize=True_future=known-future_ev-locations=full_parallel-unified': 'SOCP_parallel_uni',
 'SOCPPlanner_normalize=True_future=known-future_ev-locations=full_parallel-weird': 'SOCP_parallel_weird',
 'SOCPPlanner_normalize=True_future=no-future_ev-locations=full_None': 'SOCP_no-future',
 }


In [3]:
mosek_params = {'basisRelTolS': 1e-9,
                'basisTolS': 1e-9, 
                'intpntCoTolDfeas': 1e-9}
obj_factors = [5e5, 1e3, 1e1]
planner_full = SOCPPlanner(observe_ev_locations='full', future_model='known-future', 
                           n_future_samples=5, grid_transformation=None,
                           normalize=True,  obj_factors=obj_factors, tee=False, **mosek_params)
planner_locations_blind = SOCPPlanner(observe_ev_locations='blind', future_model='known-future', 
                                        n_future_samples=5, grid_transformation=None,
                                        normalize=True,  obj_factors=obj_factors, tee=False, **mosek_params)
planner_single_node = SOCPPlanner(observe_ev_locations='full', future_model='known-future', 
                                  n_future_samples=5, grid_transformation='single-node',
                                  normalize=True,  obj_factors=obj_factors, tee=False, 
                                  use_weird_sur_grid=False, **mosek_params)
planner_parallel = SOCPPlanner(observe_ev_locations='full', future_model='known-future', 
                               n_future_samples=5, grid_transformation='parallel',
                               normalize=True,  obj_factors=obj_factors, tee=False, 
                               use_weird_sur_grid=False, **mosek_params)

planners = [\
            planner_full,
            planner_locations_blind,
            planner_single_node, 
            planner_parallel,
           ]

In [8]:
ptu_int = 30
experiment_folder = 'IEEE16'
experiment_name = 'IEEE16meshed_con'
grid_name = 'IEEE16meshed_con_g=15_i-line-max=40_p-gen-min=-14444'

In [9]:
path_to_experiments = path_to_project + '/experiments/%s/' % experiment_folder
grid_names_list = os.listdir(path_to_experiments + '/%s/' % experiment_name)
scenarios, scenario_generator = load_scenarios(experiment_name, grid_name, path_to_experiments, ptu_int)
grid = load_grid(experiment_name, grid_name, path_to_experiments)

results_dict = {}
sc_ind = 0
scenario = scenarios[sc_ind]

In [10]:
for planner in planners:
    print('Planner', planner.name)
    env = GridEnv(grid, scenario, scenario_generator)
    results_dict[planner.name] = simulate(env, planner, tee=False)

Planner SOCPPlanner_normalize=True_future=known-future_ev-locations=full_None
Planner SOCPPlanner_normalize=True_future=known-future_ev-locations=blind_None
Planner SOCPPlanner_normalize=True_future=known-future_ev-locations=full_single-node
Planner SOCPPlanner_normalize=True_future=known-future_ev-locations=full_parallel


In [11]:
t = 31
df = []
for n_ind in range(grid.n_nodes):
    row = []
    for pl in planners:
        row.append(results_dict[pl.name]['V_nodes'][n_ind, t].round(2))
        row.append(results_dict[pl.name]['P_nodes'][n_ind, t].round(0))
        row.append(results_dict[pl.name]['P_nodes_plan'][n_ind, t].round(0))
    df.append(row)
    
    
columns = ['voltage (V)', 'p (W)', 'planned p (W)'] * len(planners)
df = pd.DataFrame(df, columns=columns)
df.index = [n.name for n in grid.nodes]
df


,voltage (V),p (W),planned p (W),voltage (V),p (W),planned p (W),voltage (V),p (W),planned p (W),voltage (V),p (W),planned p (W)
Feeder A,400.00,-13487.0,NaN,400.00,-13460.0,0.0,400.00,-13518.0,0.0,400.00,-13487.0,0.0
Feeder B,400.00,-5964.0,NaN,400.00,-5876.0,0.0,400.00,-6054.0,0.0,400.00,-5964.0,0.0
Feeder C,400.00,-13600.0,NaN,400.00,-13586.0,0.0,400.00,-13717.0,0.0,400.00,-13600.0,0.0
load_4,397.75,10000.0,NaN,397.76,10000.0,-0.0,397.75,10000.0,0.0,397.75,10000.0,0.0
load_5,398.14,10000.0,NaN,398.15,10000.0,-0.0,398.13,10000.0,0.0,398.14,10000.0,0.0
load_6,396.79,10000.0,NaN,396.80,10000.0,0.0,396.78,10000.0,0.0,396.79,10000.0,0.0
load_7,395.10,10000.0,NaN,395.11,10000.0,0.0,395.09,10000.0,0.0,395.10,10000.0,0.0
load_8,399.01,0.0,NaN,399.02,0.0,0.0,398.99,0.0,0.0,399.01,0.0,0.0
load_9,398.54,10000.0,NaN,398.58,10000.0,0.0,398.52,10000.0,0.0,398.54,10000.0,0.0
load_10,398.48,10000.0,NaN,398.48,10000.0,0.0,398.46,10000.0,0.0,398.48,10000.0,0.0


In [27]:
df.to_csv('example_opf.csv')